In [ ]:
from pd2image import convert_from_path
import pandas as pd

### Get path to pdf and start conversion process

In [ ]:
pdfs = r'"C:\User\Path_to_PDF.pdf"'
pages = convert_from_path(pdfs, 350)

Convert each page on pdf document to jpg image

In [ ]:
i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"
    page.save(image_name, "JPEG")
    i = i+1

In [ ]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt

### Mark/specify regions on image that OCR Engine would extract info 

In [ ]:
def mark_region(image_path):
    
    image = cv2.imread(image_path)
    
    # Define threshold of regions to ignore
    THRESHOLD_REGION_IGNORE = 40
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
    
    # Dilate to combine adjascent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)
    
    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    line_items_coordinates = []
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)
        
        if w < THRESHOLD_REGION_IGNORE or h < THRESHOLD_REGION_IGNORE:
            continue
            
        image = cv2.rectangle(image, (x,y), (x+w, y+h), color=(255,0,255), thickness=3)
        line_items_coordinates.append([(x,y), (x+w, y+h)])
        
    return image, line_items_coordinates

### Select image pages of PDF converted to image and apply above method

In [ ]:
FILENAME = "C\User\Path_to_image_Page_1.jpg"
image, line_items_coordinates = mark_region(FILENAME)
plt.figure(figsize=(20,20))
plt.imshow(image)

### View coordinates marked

In [ ]:
print(line_items_coordinates)

In [ ]:
### Import PyTesseract OCR Engine

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C\User\Path_to_tesseract.exe'

### Load the original image and loop through and print marked regions

In [ ]:
image = cv2.imread('Page_1.jpg')

In [ ]:
no_of_coordinates = len(line_items_coordinates)
for coordinate_point in range(no_of_coordinates):
    #get co-ordinates to crop the image
    c = line_items_coordinates[coordinate_point]
    # cropping image img = image[y0:y1, x0:x1]
    img = image[c[0][1]:c[1][1], c[0][0]:c[1][0]]
    plt.figure(figsize=(10,10))
    plt.imshow(img)